In [6]:
import numpy as np
import pandas as pd

import time
import datetime
import threading
import redis

import talib
import tushare as ts
from utils.time_util import get_market_date
from utils.stat_util import get_stat_info

red = redis.Redis(host='localhost')

In [7]:
ohlc_dict = {'open':'first','high':'max','low':'min','close':'last','vol':'sum','amount':'sum'}
def resample_freq(df_buffer, freq):
    df_freq = None
    for ticker, group in df_buffer.groupby(['code']):
        group = group.set_index(['datetime']).astype(float)
        df_temp = group.resample(freq, how=ohlc_dict, closed='right', label='right')
        df_temp['code'] = ticker
        if df_freq is None:
            df_freq = df_temp
        else:
            df_freq = pd.concat([df_freq, df_temp])
    df_freq = df_freq.dropna().reset_index()
    return df_freq[['code','datetime','open','high','low','close','vol','amount']]

In [13]:
def run_resample_freq(freq):
    intfreq = int(freq[:-3])
    cur_date = datetime.datetime.now().strftime('%Y-%m-%d')
    resample_time = '09:30:00'
    end_time = '15:00:00'
    key_1min = 't_1min_'+cur_date
    key_freq = 't_'+freq+'_'+cur_date
    df_1min, df_freq = None, None
    df_bytes_freq = red.get(key_freq)
    if df_bytes_freq is not None:
        df_freq = pd.read_msgpack(df_bytes_freq)
        resample_time = df_freq['datetime'].iloc[-1] - datetime.timedelta(minutes=intfreq)
        resample_time = resample_time.strftime('%H:%M:%S')
    while resample_time < end_time:
        df_bytes_1min = red.get(key_1min)
        df_1min = pd.read_msgpack(df_bytes_1min)
        df_1min = df_1min.loc[df_1min['datetime']>cur_date+' '+resample_time]
        if len(df_1min) == 0 and datetime.datetime.now().strftime('%Y-%m-%d') > end_time:
            print('the market has closed!')
            break
        df_cur_freq = resample_freq(df_1min, freq)
        if df_freq is None:
            df_freq = df_cur_freq
        else:
            df_freq = df_freq[df_freq['datetime']<cur_date+' '+resample_time]
            df_freq = pd.concat([df_freq, df_cur_freq])
        df_bytes_freq = df_freq.to_msgpack()
        red.set(key_freq, df_bytes_freq)
        print(df_1min['datetime'].iloc[-1].strftime('%H:%M:%S') + '  has resample into table ' + key_freq + '!')
        print('handled over, sleep {} min!'.format(intfreq*0.8))
        time.sleep(intfreq*0.8*60)
        resample_time = df_freq['datetime'].iloc[-1]
        if resample_time > df_1min['datetime'].iloc[-1]:
            resample_time = resample_time - datetime.timedelta(minutes=intfreq)
        resample_time = resample_time.strftime('%H:%M:%S')

In [15]:
run_resample_freq('15min')

/Users/harry/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The read_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.
  """Entry point for launching an IPython kernel.
/Users/harry/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  
/Users/harry/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: to_msgpack is deprecated and will be removed in a future version.
It is recommended to use pyarrow for on-the-wire transmission of pandas objects.


11:30:00  has resample into table t_15min_2020-03-03!
handled over, sleep 12.0 min!
the market has closed!
